### DEPENDENCIES

In [ ]:
import multiprocessing

from datasets import load_dataset

### Utility functions

In [ ]:
def print_length_correlation(data):
    if data["attention_mask"]:
        print("input_ids -", len(data["input_ids"]))
        print("attention_mask -", len(data["attention_mask"]))
        print("labels -", len(data["labels"]))
    else:
        print("input_ids -", len(data["input_ids"]))
        print("labels -", len(data["labels"]))

### Data preprocessing

In [ ]:
# loading the dataset
PII = load_dataset("ai4privacy/pii-masking-400k")

# splitting the dataset into train and validation
#ds_t = ds_raw["train"]
#ds_v = ds_raw["validation"]

# extracting English data points
cores = multiprocessing.cpu_count()

# how to run them concurrently?
PII = PII.filter(lambda example: example["language"] == "en", num_proc = cores)

# remove unnecessary columns
PII = PII.remove_columns(["locale", "language", "split", "uid"])

# shuffle the train dataset
PII = PII.shuffle(seed=33)

# reduce the train dataset
select_train = int(0.10 * len(PII["train"]))
select_validation = int(0.10 * len(PII["validation"]))
PII["train"] = PII["train"].select(range(select_train))
PII["validation"] = PII["validation"].select(range(select_validation))

In [ ]:
PII

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-cased")

### Adding special tokens([CLS] and [SEP]) to "mbert_tokens"

In [ ]:
def add_special_tokens(example):
    example["mbert_tokens"].insert(0, "[CLS]")
    example["mbert_tokens"].append("[SEP]")
    return example

PII = PII.map(add_special_tokens)

### Creating "input_ids" for "mbert_tokens"

In [ ]:
def create_ids(example):
    example["input_ids"] = tokenizer.convert_tokens_to_ids(example["mbert_tokens"])
    return example

PII = PII.map(create_ids)

### Creating attention masks

In [ ]:
def create_attention_masks(example):
    example["attention_mask"] = [1] * (len(example["mbert_tokens"]))
    return example

PII = PII.map(create_attention_masks)

In [ ]:
PII

### Getting available NER classes and assigning unique NER class IDs

In [ ]:
ner_classes = []
ner_ids = {}
ID = 0

# getting available NER classes
for row in PII["train"]:
   for token_class in row["mbert_token_classes"]:
      if token_class not in ner_classes:
         ner_classes.append(token_class)

# assigning unique IDs
for ner_class in ner_classes:
   ner_ids[ner_class] = None

for ner_class in ner_ids.keys():
   ner_ids[ner_class] = ID
   ID+=1

In [ ]:
print(ner_classes)
print(ner_ids)

### Creating labels for NER classes

In [ ]:
def create_label_list(token_classes):
    labels = []
    for token_class in token_classes:
        labels.append(ner_ids[token_class])
    return labels

def create_labels(example):
    example["labels"] = create_label_list(example["mbert_token_classes"])
    return example

PII = PII.map(create_labels)

In [ ]:
PII

In [ ]:
print_length_correlation(PII["train"][0])

In [ ]:
def add_special_labels(example):
    example["labels"].insert(0, -100)
    example["labels"].append(-100)
    return example

PII = PII.map(add_special_labels)

In [ ]:
print_length_correlation(PII["train"][0])

In [ ]:
# find the longest input_id

def longest_id(data):
    max_length = 0
    for row in data["input_ids"]:
        if len(row) > max_length:
            max_length = len(row)
    return max_length

In [ ]:
print(longest_id(PII["train"]))

In [ ]:
print(longest_id(PII["validation"]))

In [ ]:
def pad(examples):
    max_length = 512
    return tokenizer.pad(examples, padding="max_length", max_length=max_length) 

PII = PII.map(pad, batched=True)

In [ ]:
print_length_correlation(PII["train"][0])

In [ ]:
def pad_labels(example):
    labels_len = len(example["labels"])
    input_ids_len = len(example["input_ids"]) 
    if labels_len < input_ids_len:
        for i in range(input_ids_len - labels_len):
            example["labels"].append(-100) 
    return example

PII = PII.map(pad_labels)

In [ ]:
print_length_correlation(PII["train"][0])

In [ ]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
label2id = ner_ids
print(label2id)

In [ ]:
id2label = {}
for value, key in enumerate(ner_ids):
    id2label[value] = key

print(id2label)

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    "distilbert/distilbert-base-cased", num_labels=35, id2label=id2label, label2id=label2id
)

In [ ]:
training_args = TrainingArguments(
    output_dir="OG_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=PII["train"],
    eval_dataset=PII["validation"],
    processing_class=tokenizer
)

trainer.train()